In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn notebook xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import re
import xgboost as xgb

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
plt.style.use('ggplot')

print("Bibliotecas importadas com sucesso.")


Bibliotecas importadas com sucesso.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os

BASE_DADOS_ANON = "/content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados"
BASE_OUT = "/content/drive/MyDrive/Mestrado/Artigo - Dados/Resultados_Modelos"

os.makedirs(BASE_OUT, exist_ok=True)

ANOS = [2018, 2019, 2020, 2021, 2022, 2023]

TECNICAS = {
    "original":           f"{BASE_DADOS_ANON}/original",
    "permutacao":         f"{BASE_DADOS_ANON}/permutacao",
    "generalizacao_dec1": f"{BASE_DADOS_ANON}/generalizacao_dec1",
    "microagregacao_k5":  f"{BASE_DADOS_ANON}/microagregacao_k5",
    "k_anon_k2":          f"{BASE_DADOS_ANON}/k_anon_k2",
    "dp_eps_1.0":         f"{BASE_DADOS_ANON}/dp/eps_1.0",
}

print("BASE_DADOS_ANON:", BASE_DADOS_ANON)
print("BASE_OUT      :", BASE_OUT)
print("Técnicas:", list(TECNICAS.keys()))


BASE_DADOS_ANON: /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados
BASE_OUT      : /content/drive/MyDrive/Mestrado/Artigo - Dados/Resultados_Modelos
Técnicas: ['original', 'permutacao', 'generalizacao_dec1', 'microagregacao_k5', 'k_anon_k2', 'dp_eps_1.0']


In [5]:
def carregar_mosquito_por_tecnica(pasta_tecnica, anos=ANOS):
    files_to_load = [f"{pasta_tecnica}/{ano}.csv" for ano in anos]

    print(f"Buscando arquivos da técnica em: {pasta_tecnica}")
    all_mosquito_dfs = []

    for file in files_to_load:
        found = glob(file)
        if found:
            filepath = found[0]
            try:
                print(f"Lendo {filepath}...")
                df_year = pd.read_csv(
                    filepath,
                    sep=',',              # <- AJUSTE para seus CSVs do Passo 1
                    low_memory=False,
                    dtype={'inspection_details_device_id': str,
                           'inspection_mosquitoes_0_pivot_serial': str}
                )
                all_mosquito_dfs.append(df_year)
            except Exception as e:
                print(f"Erro ao ler {filepath}: {e}")
        else:
            print(f"AVISO: Arquivo {file} não encontrado!")

    if all_mosquito_dfs:
        df_mosquito_raw = pd.concat(all_mosquito_dfs, ignore_index=True)
        print("\n--- SUCESSO! ---")
        print(f"Dados carregados ({anos[0]}-{anos[-1]}) para técnica.")
        print(f"Total de registros: {len(df_mosquito_raw)}")
        print("Amostra de coordenadas:")
        print(df_mosquito_raw[['latitude', 'longitude']].head(3))
        return df_mosquito_raw
    else:
        raise RuntimeError("ERRO CRÍTICO: Nenhum dado foi carregado.")


In [6]:
climate_files = sorted(glob('/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_*.CSV'))

if not climate_files:
    print("Erro: Nenhum arquivo INMET_*.CSV encontrado.")
    print("Por favor, faça o upload dos arquivos de clima.")
else:
    print(f"Arquivos de clima encontrados: {climate_files}")

all_climate_dfs = []
for file in climate_files:
    try:
        print(f"Lendo {file}...")
        df_c = pd.read_csv(
            file,
            sep=';',
            skiprows=8,
            decimal=',',
            encoding='latin-1'
        )
        all_climate_dfs.append(df_c)
    except Exception as e:
        print(f"Erro ao ler {file}: {e}")

if all_climate_dfs:
    df_climate_raw = pd.concat(all_climate_dfs, ignore_index=True)
    print("\nDados de clima de 2018 a 2023 combinados com sucesso!")
    print(f"Total de registros de clima (horários): {len(df_climate_raw)}")
else:
    print("Nenhum dado de clima foi carregado.")


Arquivos de clima encontrados: ['/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2018_A_31-12-2018.CSV', '/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2019_A_31-12-2019.CSV', '/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2020_A_31-12-2020.CSV', '/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2021_A_31-12-2021.CSV', '/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_B807_PORTO ALEGRE- BELEM NOVO_01-01-2023_A_31-12-2023.CSV', '/content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_B807_PORTO ALEGRE- BELEM NOVO_08-12-2022_A_31-12-2022.CSV']
Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2018_A_31-12-2018.CSV...
Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2019_A_31-12-2019.CSV...
Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/INMET_S_RS_A801_PORTO ALEGRE_01-01-2020_A

In [7]:
if 'df_climate_raw' in locals():
    print("Limpando e processando dados de clima...")
    df_climate = df_climate_raw.copy()

    print(f"Colunas originais encontradas: {df_climate.columns.to_list()}")

    rename_map = {}
    for col in df_climate.columns:
        col_cleaned = col.strip()
        if 'DATA (YYYY-MM-DD)' in col_cleaned:
            rename_map[col] = 'data_v1'
        elif col_cleaned == 'Data':
            rename_map[col] = 'data_v2'
        elif 'HORA (UTC)' in col_cleaned:
            rename_map[col] = 'hora_v1'
        elif col_cleaned == 'Hora UTC':
            rename_map[col] = 'hora_v2'
        elif 'PRECIPITAÇÃO' in col_cleaned:
            rename_map[col] = 'precipitacao_mm'
        elif 'TEMPERATURA DO AR' in col_cleaned:
            rename_map[col] = 'temp_c'
        elif 'TEMPERATURA MÍNIMA' in col_cleaned:
            rename_map[col] = 'temp_min_c'
        elif 'TEMPERATURA MÁXIMA' in col_cleaned:
            rename_map[col] = 'temp_max_c'
        elif 'UMIDADE RELATIVA' in col_cleaned:
            rename_map[col] = 'umidade_rel'

    df_climate.rename(columns=rename_map, inplace=True)
    print("Colunas de clima renomeadas para nomes provisórios (v1, v2).")

    df_climate['data'] = df_climate['data_v1'].combine_first(df_climate['data_v2'])
    df_climate['hora'] = df_climate['hora_v1'].combine_first(df_climate['hora_v2'])

    if df_climate['data'].isnull().all():
        print("ERRO CRÍTICO: Consolidação da coluna 'data' falhou.")
    else:
        print("Colunas 'data' e 'hora' consolidadas com sucesso.")

    df_climate['data_str'] = df_climate['data'].astype(str).str.replace('-', '/')
    df_climate['hora_str'] = df_climate['hora'].astype(str).str.replace(' UTC', '').str.zfill(4)

    df_climate['datetime'] = pd.to_datetime(
        df_climate['data_str'] + ' ' + df_climate['hora_str'],
        format='%Y/%m/%d %H%M',
        errors='coerce'
    )

    df_climate = df_climate.dropna(subset=['datetime'])

    cols_interesse = ['datetime', 'precipitacao_mm', 'temp_c', 'temp_min_c', 'temp_max_c', 'umidade_rel']
    cols_existentes = [col for col in cols_interesse if col in df_climate.columns]

    df_climate_clean = df_climate[cols_existentes].set_index('datetime')

    for col in df_climate_clean.columns:
        df_climate_clean[col] = pd.to_numeric(df_climate_clean[col], errors='coerce')

    df_climate_clean = df_climate_clean.groupby(df_climate_clean.index).mean()

    print("Agregando dados de clima por semana...")
    df_climate_weekly = df_climate_clean.resample('W').agg(
        precip_total_mm=('precipitacao_mm', 'sum'),
        temp_media_c=('temp_c', 'mean'),
        temp_min_c=('temp_min_c', 'min'),
        temp_max_c=('temp_max_c', 'max'),
        umidade_media_rel=('umidade_rel', 'mean')
    )

    df_climate_weekly = df_climate_weekly.interpolate(method='time')

    print("DataFrame de CLIMA semanal criado com sucesso.")
    print(df_climate_weekly.head())
else:
    print("Pule Célula 4: df_climate_raw não foi criado.")


Limpando e processando dados de clima...
Colunas originais encontradas: ['DATA (YYYY-MM-DD)', 'HORA (UTC)', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)', 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)', 'RADIACAO GLOBAL (KJ/m²)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)', 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)', 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'UMIDADE RELATIVA DO AR, HORARIA (%)', 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)', 'Unnamed: 19', 'Data', 'Hora UTC', 'RADIACAO GLOBAL (Kj/m²)']
Colunas de clima renomeadas para nomes provisórios (v1, v2).
Colunas 'data' e 'hora' consol

In [8]:
if 'df_climate_clean' in locals():
    print("Iniciando Engenharia de Features de Clima (Diária)...")

    df_climate_daily = df_climate_clean.resample('D').agg(
        precip_total_mm=('precipitacao_mm', 'sum'),
        temp_media_c=('temp_c', 'mean'),
        temp_min_c=('temp_min_c', 'min')
    )
    df_climate_daily = df_climate_daily.interpolate(method='time')

    df_climate_daily['clima_chuva_acum_21d'] = df_climate_daily['precip_total_mm'].rolling(window=21, min_periods=1).sum()
    df_climate_daily['clima_temp_min_media_14d'] = df_climate_daily['temp_min_c'].rolling(window=14, min_periods=1).mean()

    cond_calor = df_climate_daily['temp_min_c'] > 20
    dias_calor = cond_calor.astype(int).groupby(cond_calor.eq(0).cumsum()).cumsum()
    df_climate_daily['clima_dias_temp_min_20c'] = dias_calor

    cond_chuva = df_climate_daily['precip_total_mm'] > 0
    dias_sem_chuva = cond_chuva.eq(0).astype(int).groupby(cond_chuva.astype(int).cumsum()).cumsum()
    df_climate_daily['clima_dias_sem_chuva'] = dias_sem_chuva

    features_biologicas = [
        'clima_chuva_acum_21d',
        'clima_temp_min_media_14d',
        'clima_dias_temp_min_20c',
        'clima_dias_sem_chuva'
    ]

    df_climate_features_weekly = df_climate_daily[features_biologicas].resample('W').mean()

    print("Features de clima biológicas criadas e agregadas por semana.")
    print(df_climate_features_weekly.head())
else:
    print("Pule Célula 4a: df_climate_clean não foi criado.")


Iniciando Engenharia de Features de Clima (Diária)...
Features de clima biológicas criadas e agregadas por semana.
            clima_chuva_acum_21d  clima_temp_min_media_14d  \
datetime                                                     
2019-01-06              8.833333                 22.561944   
2019-01-13             18.085714                 22.324900   
2019-01-20             62.400000                 22.469388   
2019-01-27             91.285714                 21.981633   
2019-02-03             99.400000                 22.607143   

            clima_dias_temp_min_20c  clima_dias_sem_chuva  
datetime                                                   
2019-01-06                 1.500000              1.000000  
2019-01-13                 6.000000              0.571429  
2019-01-20                 8.571429              0.000000  
2019-01-27                 4.000000              1.714286  
2019-02-03                 9.000000              2.142857  


In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, RocCurveDisplay

def rodar_tecnica_e_salvar(tecnica_nome, pasta_tecnica):
    out_dir = f"{BASE_OUT}/{tecnica_nome}"
    os.makedirs(out_dir, exist_ok=True)

    # ----------------------------
    # CARGA MOSQUITO (por técnica)
    # ----------------------------
    df_mosquito_raw = carregar_mosquito_por_tecnica(pasta_tecnica)

    # ----------------------------
    # CÉLULA 5 DO BRUNO (mosquito)
    # ----------------------------
    if 'df_mosquito_raw' in locals():
        print("Limpando e processando dados de mosquito...")
        df_mosquito = df_mosquito_raw.copy()

        df_mosquito['data_inspecao'] = pd.to_datetime(
            df_mosquito['inspection_realized_at'],
            format='%d/%m/%Y %H:%M',
            errors='coerce'
        )

        # robustez para decimal com vírgula (não altera quando já é float)
        df_mosquito['latitude'] = df_mosquito['latitude'].astype(str).str.replace(',', '.', regex=False)
        df_mosquito['longitude'] = df_mosquito['longitude'].astype(str).str.replace(',', '.', regex=False)
        df_mosquito['latitude'] = pd.to_numeric(df_mosquito['latitude'], errors='coerce')
        df_mosquito['longitude'] = pd.to_numeric(df_mosquito['longitude'], errors='coerce')

        df_mosquito['total_aedes_aegypti'] = 0

        for i in range(10):
            name_col = f'inspection_mosquitoes_{i}_name'
            qty_col  = f'inspection_mosquitoes_{i}_pivot_quantity'

            if name_col in df_mosquito.columns and qty_col in df_mosquito.columns:
                df_mosquito[qty_col] = pd.to_numeric(df_mosquito[qty_col], errors='coerce').fillna(0)

                condicao_aedes = (df_mosquito[name_col] == 'Aedes aegypti')
                df_mosquito.loc[condicao_aedes, 'total_aedes_aegypti'] += df_mosquito[qty_col]

        df_mosquito_clean = df_mosquito[
            ['data_inspecao', 'latitude', 'longitude', 'total_aedes_aegypti']
        ].copy()

        df_mosquito_clean = df_mosquito_clean.dropna(subset=['data_inspecao', 'latitude', 'longitude'])

        print("Dados de mosquito limpos e contagem de Aedes concluída.")
        print(f"Total de inspeções válidas (com data e geo): {len(df_mosquito_clean)}")
        print(f"Total de Aedes aegypti capturados (2018-2023): {df_mosquito_clean['total_aedes_aegypti'].sum()}")
    else:
        raise RuntimeError("Pule Célula 5: df_mosquito_raw não foi criado.")

    # ----------------------------
    # CÉLULA 6 DO BRUNO (clusters) - versão robusta
    # ----------------------------
    if 'df_mosquito_clean' in locals():
        print("Iniciando a criação de clusters espaciais (regiões)...")

        # 0) Garantir tipos e remover inválidos
        df_tmp = df_mosquito_clean.copy()
        df_tmp['latitude']  = pd.to_numeric(df_tmp['latitude'], errors='coerce')
        df_tmp['longitude'] = pd.to_numeric(df_tmp['longitude'], errors='coerce')
        df_tmp = df_tmp.dropna(subset=['latitude', 'longitude'])

        # 1) Coordenadas únicas (uma por armadilha/local)
        df_geo_unique = df_tmp[['latitude', 'longitude']].drop_duplicates().copy()

        # 2) (Opcional, mas recomendado) remover outliers geográficos
        #    Mantém o KMeans e o plot coerentes; ajuste se seu estudo não for POA.
        df_geo_unique = df_geo_unique[
            df_geo_unique["latitude"].between(-30.20, -29.80) &
            df_geo_unique["longitude"].between(-51.35, -50.95)
        ].copy()

        # 3) Ajuste automático do número de clusters (evita erro quando a técnica colapsa pontos únicos)
        N_REGIOES_DESEJADO = 15
        n_pontos = len(df_geo_unique)

        if n_pontos < 2:
            raise RuntimeError(f"{tecnica_nome}: pontos únicos insuficientes para clusterizar (n={n_pontos}).")

        N_REGIOES = min(N_REGIOES_DESEJADO, n_pontos)
        if N_REGIOES != N_REGIOES_DESEJADO:
            print(f"⚠️ {tecnica_nome}: apenas {n_pontos} pontos únicos. Ajustando N_REGIOES: {N_REGIOES_DESEJADO} -> {N_REGIOES}")

        # 4) KMeans
        scaler = StandardScaler()
        coords_scaled = scaler.fit_transform(df_geo_unique[['latitude', 'longitude']])

        kmeans = KMeans(n_clusters=N_REGIOES, random_state=42, n_init=10)
        df_geo_unique['regiao_cluster'] = kmeans.fit_predict(coords_scaled)

        print(f"Agrupando armadilhas em {N_REGIOES} regiões...")

        # 5) Plot com zoom e aspecto correto (melhor visual)
        plt.figure(figsize=(10, 8))
        ax = sns.scatterplot(
            data=df_geo_unique,
            x='longitude',
            y='latitude',
            hue='regiao_cluster',
            palette='tab20',
            legend='full',
            s=10
        )

        # zoom automático
        margin_lon = 0.005
        margin_lat = 0.005
        plt.xlim(df_geo_unique["longitude"].min() - margin_lon, df_geo_unique["longitude"].max() + margin_lon)
        plt.ylim(df_geo_unique["latitude"].min() - margin_lat, df_geo_unique["latitude"].max() + margin_lat)

        # aspecto geográfico
        ax.set_aspect('equal', adjustable='box')

        plt.title(f'Armadilhas Agrupadas por Região (Clusters K-Means) - {tecnica_nome}')
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.savefig(f"{out_dir}/clusters.png", dpi=200, bbox_inches="tight")
        plt.close()

        # 6) Merge: adiciona a região em todas as inspeções
        df_mosquito_final = pd.merge(
            df_tmp,  # usa df_tmp (limpo)
            df_geo_unique[['latitude', 'longitude', 'regiao_cluster']],
            on=['latitude', 'longitude'],
            how='left'
        )

        print("Coluna 'regiao_cluster' adicionada a todas as inspeções.")
        print(df_mosquito_final.head())

    else:
        raise RuntimeError("Pule Célula 6: df_mosquito_clean não foi criado.")


      # ----------------------------
      # CÉLULA 7 DO BRUNO (master weekly) - CORRIGIDA
    # ----------------------------
    if 'df_mosquito_final' in locals():
        if 'df_climate_features_weekly' not in globals():
            raise RuntimeError("ERRO: df_climate_features_weekly não está definido. Rode as células de CLIMA antes do loop.")

        print("Iniciando criação do DataFrame Mestre Semanal...")

        df_master = df_mosquito_final.set_index('data_inspecao')

        df_city_weekly = df_master['total_aedes_aegypti'].resample('W').sum().to_frame(name='X_total_cidade')

        df_regional_weekly = df_master.groupby('regiao_cluster').resample('W')['total_aedes_aegypti'].sum()
        df_regional_pivot = df_regional_weekly.unstack(level=0).fillna(0)
        df_regional_pivot.columns = [f'X_total_regiao_{int(c)}' for c in df_regional_pivot.columns]

        df_master_weekly = pd.merge(df_city_weekly, df_regional_pivot, left_index=True, right_index=True, how='left')
        df_master_weekly = pd.merge(df_master_weekly, df_climate_features_weekly, left_index=True, right_index=True, how='left')
        df_master_weekly = df_master_weekly.fillna(0)

        print("DataFrame Mestre Semanal (v2) criado com sucesso!")
        print(f"Shape: {df_master_weekly.shape}")
        print(df_master_weekly.head())
    else:
        raise RuntimeError("Pule Célula 7: df_mosquito_final não foi criado.")


    # ----------------------------
    # CÉLULA 8 DO BRUNO (limiar + alvo)
    # ----------------------------
    if 'df_master_weekly' in locals():
        limiar_surto = df_master_weekly.loc[df_master_weekly.index.year < 2023, 'X_total_cidade'].quantile(0.80)
        print(f"Limiar do 'Surto' (Percentil 80, dados de treino): {limiar_surto:.2f} mosquitos/semana")

        df_master_weekly['Y_surto_na_semana'] = (df_master_weekly['X_total_cidade'] > limiar_surto).astype(int)

        plt.figure(figsize=(20, 8))
        df_master_weekly['X_total_cidade'].plot(label='Capturas Semanais (Cidade)')
        plt.axhline(y=limiar_surto, color='r', linestyle='--', label=f'Limiar do Surto (P80 = {limiar_surto:.0f})')
        plt.title(f'Definição do Limiar de Surto - {tecnica_nome}')
        plt.legend()
        plt.savefig(f"{out_dir}/limiar_surto.png", dpi=200, bbox_inches="tight")
        plt.close()

        print("Distribuição da nossa variável alvo (em todo o período):")
        print(df_master_weekly['Y_surto_na_semana'].value_counts(normalize=True))
    else:
        raise RuntimeError("Pule Célula 8: df_master_weekly não foi criado.")

    # ----------------------------
    # CÉLULA 9 DO BRUNO (lags)
    # ----------------------------
    if 'df_master_weekly' in locals() and 'Y_surto_na_semana' in df_master_weekly.columns:
        print("Criando features de lag (defasagem)...")

        df_model = df_master_weekly.copy()
        feature_cols = [col for col in df_model.columns if col.startswith('X_') or col.startswith('clima_')]

        print(f"Features de base para criar lags: {feature_cols}")

        for col in feature_cols:
            for lag in range(1, 5):
                df_model[f'{col}_lag_{lag}'] = df_model[col].shift(lag)

        for col in feature_cols:
            df_model[f'{col}_mm_4sem'] = df_model[col].shift(1).rolling(window=4).mean()

        df_model = df_model.dropna()

        print("DataFrame pronto para o modelo (com features 'clima_')")
        print(f"Shape final do df_model: {df_model.shape}")
    else:
        raise RuntimeError("Pule Célula 9: df_master_weekly não foi processado.")

    # ----------------------------
    # CÉLULA 10 DO BRUNO (features lag >=2)
    # ----------------------------
    if 'df_model' in locals():
        print("--- Lógica Preditiva (lag >= 2) com Features Biológicas ---")

        features_mosquito = [
            col for col in df_model.columns
            if col.startswith('X_')
            and ('_lag_' in col)
            and ('_lag_1' not in col)
            and ('_mm_' not in col)
        ]

        features_clima = [
            col for col in df_model.columns
            if col.startswith('clima_')
            and ('_lag_' in col)
            and ('_lag_1' not in col)
            and ('_mm_' not in col)
        ]

        features = features_mosquito + features_clima
        target = 'Y_surto_na_semana'

        print(f"Total de features de mosquito (lags >= 2): {len(features_mosquito)}")
        print(f"Total de features de clima (lags >= 2): {len(features_clima)}")
        print(f"Total de features para o modelo: {len(features)}")

        train_df = df_model[df_model.index.year < 2023]
        X_train = train_df[features]
        y_train = train_df[target]

        test_df = df_model[df_model.index.year == 2023]
        X_test = test_df[features]
        y_test = test_df[target]

        print(f"Tamanho do Treino: {X_train.shape[0]} semanas (2018-2022)")
        print(f"Tamanho do Teste: {X_test.shape[0]} semanas (2023)")

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    else:
        raise RuntimeError("Pule Célula 10: df_model não foi criado.")

    # ----------------------------
    # CÉLULA 11 DO BRUNO (XGBoost)
    # ----------------------------
    if 'X_train_scaled' in locals():
        contagem_negativa = (y_train == 0).sum()
        contagem_positiva = (y_train == 1).sum()
        scale_pos_weight_value = contagem_negativa / contagem_positiva

        print(f"--- Treinando Modelo XGBoost (v4.0) ---")
        print(f"Peso de balanceamento (scale_pos_weight) calculado: {scale_pos_weight_value:.2f}")

        model = xgb.XGBClassifier(
            n_estimators=150,
            learning_rate=0.05,
            random_state=42,
            n_jobs=-1,
            scale_pos_weight=scale_pos_weight_value
        )

        print("Treinando o modelo XGBoost...")
        model.fit(X_train_scaled, y_train)
        print("Treinamento concluído.")

        feature_importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
        plt.figure(figsize=(10, 10))
        sns.barplot(x=feature_importances.head(20), y=feature_importances.head(20).index)
        plt.title(f'Top 20 Features Mais Importantes (XGBoost) - {tecnica_nome}')
        plt.xlabel('Importância (Gain)')
        plt.ylabel('Feature')
        plt.savefig(f"{out_dir}/top20_features.png", dpi=200, bbox_inches="tight")
        plt.close()
    else:
        raise RuntimeError("Pule Célula 11: Dados de treino não encontrados.")

    # ----------------------------
    # CÉLULA 12 DO BRUNO (avaliação + AUC)
    # ----------------------------
    if 'model' in locals():
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

        print("\n--- Relatório de Classificação (Performance em 2023) ---")
        print(classification_report(y_test, y_pred, target_names=['Não-Surto (0)', 'Surto (1)']))

        print("\n--- Matriz de Confusão ---")
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Previsto Não-Surto', 'Previsto Surto'],
                    yticklabels=['Real Não-Surto', 'Real Surto'])
        plt.ylabel('Verdadeiro (Real)')
        plt.xlabel('Previsão (Modelo)')
        plt.title(f"Matriz de Confusão - {tecnica_nome}")
        plt.savefig(f"{out_dir}/cm.png", dpi=200, bbox_inches="tight")
        plt.close()

        auc = roc_auc_score(y_test, y_pred_proba)
        print(f"\nAUC Score (de 0.0 a 1.0): {auc:.4f}")

        # ROC salva (extra p/ Passo 3)
        plt.figure(figsize=(7, 6))
        RocCurveDisplay.from_predictions(y_test, y_pred_proba, name=tecnica_nome)
        plt.title(f"ROC - {tecnica_nome}")
        plt.grid(True)
        plt.savefig(f"{out_dir}/roc.png", dpi=200, bbox_inches="tight")
        plt.close()
    else:
        raise RuntimeError("Pule Célula 12: Modelo não foi treinado.")

    # ----------------------------
    # CÉLULA 13 DO BRUNO (real vs previsto 2023)
    # ----------------------------
    if 'y_pred_proba' in locals():
        results_2023 = df_model[df_model.index.year == 2023].copy()
        results_2023['previsao_modelo'] = y_pred
        results_2023['probabilidade_surto'] = y_pred_proba

        fig, ax1 = plt.subplots(figsize=(20, 10))

        color = 'tab:gray'
        ax1.set_xlabel('Data (2023)')
        ax1.set_ylabel('Captura Real (Mosquitos)', color=color)
        ax1.plot(results_2023.index, results_2023['X_total_cidade'], color=color, alpha=0.7, label='Captura Real')
        ax1.axhline(y=limiar_surto, color='red', linestyle='--', label=f'Limiar Surto (Real > {limiar_surto:.0f})')
        ax1.tick_params(axis='y', labelcolor=color)

        ax2 = ax1.twinx()
        color = 'tab:blue'
        ax2.set_ylabel('Probabilidade de Surto (Modelo)', color=color)
        ax2.plot(results_2023.index, results_2023['probabilidade_surto'], color=color, label='Probabilidade Surto (Modelo)')
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.set_ylim(0, 1)

        fig.tight_layout()
        plt.title(f'Performance do Modelo em 2023: Real vs. Previsto - {tecnica_nome}')
        plt.savefig(f"{out_dir}/real_vs_previsto_2023.png", dpi=200, bbox_inches="tight")
        plt.close()
    else:
        raise RuntimeError("Pule Célula 13: Previsões não foram geradas.")

    # ----------------------------
    # SALVAMENTO PADRONIZADO (para Passo 3)
    # ----------------------------
    # predicoes.csv
    df_pred = pd.DataFrame({
        "data": test_df.index.astype(str),
        "y_true": y_test.astype(int).values,
        "y_pred": y_pred.astype(int),
        "y_proba": y_pred_proba.astype(float)
    })
    df_pred.to_csv(f"{out_dir}/predicoes.csv", index=False)

    # metricas.csv
    met = {
        "tecnica": tecnica_nome,
        "AUC_ROC": float(auc),
        "Recall_Surto": float(recall_score(y_test, y_pred)),
        "Precision_Surto": float(precision_score(y_test, y_pred)),
        "F1_Surto": float(f1_score(y_test, y_pred)),
        "Balanced_Accuracy": float(balanced_accuracy_score(y_test, y_pred)),
        "N_teste": int(len(y_test)),
        "Positivos_teste": int(y_test.sum()),
        "limiar_surto_P80": float(limiar_surto)
    }
    pd.DataFrame([met]).to_csv(f"{out_dir}/metricas.csv", index=False)

    print(f"✅ Artefatos salvos em: {out_dir}")
    return met


In [14]:
resultados = []

for tecnica_nome, pasta in TECNICAS.items():
    try:
        met = rodar_tecnica_e_salvar(tecnica_nome, pasta)
        resultados.append(met)
    except Exception as e:
        print(f"❌ Falhou na técnica {tecnica_nome}: {e}")

df_resultados = pd.DataFrame(resultados)

if df_resultados.empty:
    raise RuntimeError("Nenhuma técnica gerou resultados. Verifique erros acima.")

if "AUC_ROC" in df_resultados.columns:
    df_resultados = df_resultados.sort_values(by="AUC_ROC", ascending=False)

resumo_path = f"{BASE_OUT}/resumo_metricas_todas_tecnicas.csv"
df_resultados.to_csv(resumo_path, index=False)

print("✅ Resumo salvo em:", resumo_path)
df_resultados



Buscando arquivos da técnica em: /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original/2018.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original/2019.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original/2020.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original/2021.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original/2022.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/original/2023.csv...

--- SUCESSO! ---
Dados carregados (2018-2023) para técnica.
Total de registros: 292248
Amostra de coordenadas:
   latitude  longitude
0   -30.066    -51.234
1   -30.032    -51.112
2   -30.009    -51.171
Limpando e processando dados de mosquito...
Dados de mosquito limpos e contagem de Aedes concluída.
Total de inspeções válidas (co

/tmp/ipython-input-1864583872.py:204: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_model[f'{col}_mm_4sem'] = df_model[col].shift(1).rolling(window=4).mean()
/tmp/ipython-input-1864583872.py:204: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_model[f'{col}_mm_4sem'] = df_model[col].shift(1).rolling(window=4).mean()


Treinamento concluído.

--- Relatório de Classificação (Performance em 2023) ---
               precision    recall  f1-score   support

Não-Surto (0)       0.50      0.71      0.59         7
    Surto (1)       0.86      0.71      0.77        17

     accuracy                           0.71        24
    macro avg       0.68      0.71      0.68        24
 weighted avg       0.75      0.71      0.72        24


--- Matriz de Confusão ---

AUC Score (de 0.0 a 1.0): 0.7479
✅ Artefatos salvos em: /content/drive/MyDrive/Mestrado/Artigo - Dados/Resultados_Modelos/original
Buscando arquivos da técnica em: /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/permutacao
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/permutacao/2018.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/permutacao/2019.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/permutacao/2020.csv...
Lendo /content/drive/MyDrive/Me

/tmp/ipython-input-1864583872.py:204: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_model[f'{col}_mm_4sem'] = df_model[col].shift(1).rolling(window=4).mean()
/tmp/ipython-input-1864583872.py:204: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_model[f'{col}_mm_4sem'] = df_model[col].shift(1).rolling(window=4).mean()


Treinamento concluído.

--- Relatório de Classificação (Performance em 2023) ---
               precision    recall  f1-score   support

Não-Surto (0)       0.60      0.43      0.50         7
    Surto (1)       0.79      0.88      0.83        17

     accuracy                           0.75        24
    macro avg       0.69      0.66      0.67        24
 weighted avg       0.73      0.75      0.74        24


--- Matriz de Confusão ---

AUC Score (de 0.0 a 1.0): 0.5714
✅ Artefatos salvos em: /content/drive/MyDrive/Mestrado/Artigo - Dados/Resultados_Modelos/permutacao
Buscando arquivos da técnica em: /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/generalizacao_dec1
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/generalizacao_dec1/2018.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/generalizacao_dec1/2019.csv...
Lendo /content/drive/MyDrive/Mestrado/Artigo - Dados/Dados_Anonimizados/generalizacao_dec1/2020.csv.

...

<Figure size 700x600 with 0 Axes>

<Figure size 700x600 with 0 Axes>

<Figure size 700x600 with 0 Axes>

<Figure size 700x600 with 0 Axes>

<Figure size 700x600 with 0 Axes>